In [1]:
# pyscience imports
import os
import sys
import glob
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-darkgrid")
# plt.style.use("dark_background")
sns.set(style="ticks", context="talk")
# %matplotlib inline
# run for jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
#%%


In [2]:
%pwd

'/mnt/d/Temp/football_table_scraper'

In [3]:
eighties = list(range(1980, 1991, 1))

In [23]:
english_league = []
scottish_league = []
for s in eighties:
    # english league
    try:
        r = pd.read_csv('english_league_tables/english_league_' + str(s) + '.csv')
        r['n_teams'] = len(r)
        english_league.append(r)
    except:
        print('English League ' + str(s) + ' league table not found')
    # scottish league
    try:
        r = pd.read_csv('scottish_league_tables/scottish_league_' + str(s) + '.csv')
        r['n_teams'] = len(r)
        scottish_league.append(r)
    except:
        print('Scottish League ' + str(s) + ' league table not found')

In [24]:
spl_1980s = pd.concat(scottish_league)
spl_1980s = spl_1980s.iloc[:, 1:]
spl_1980s.head()

,club,played,home_w,home_d,home_l,home_f,home_a,away_w,away_d,away_l,away_f,away_a,pts,year,n_teams
0,Aberdeen,36.0,10,4,4.0,30.0,18.0,9.0,6.0,3.0,38.0,18.0,48.0,1980,10
1,Celtic,36.0,13,3,2.0,44.0,17.0,5.0,8.0,5.0,17.0,21.0,47.0,1980,10
2,St. Mirren,36.0,11,5,2.0,37.0,23.0,4.0,7.0,7.0,19.0,26.0,42.0,1980,10
3,Dundee United,36.0,9,7,2.0,23.0,6.0,3.0,6.0,9.0,20.0,24.0,37.0,1980,10
4,Rangers,36.0,11,5,2.0,29.0,16.0,4.0,2.0,12.0,21.0,30.0,37.0,1980,10


In [25]:
epl_1980s = pd.concat(english_league)
epl_1980s = epl_1980s.iloc[:, 1:]
epl_1980s.head()

,Pos,Team,Played,home_w,home_d,home_l,home_f,home_a,away_w,away_d,away_l,away_f,away_a,Pts,year,n_teams
0,1,LIVERPOOL,42,15,6,0,46,8,10,4,7,35,22,60,80,22
1,2,Manchester United,42,17,3,1,43,8,7,7,7,22,27,58,80,22
2,3,Ipswich Town,42,14,4,3,43,13,8,5,8,25,26,53,80,22
3,4,Arsenal,42,8,10,3,24,12,10,6,5,28,24,52,80,22
4,5,Nottingham Forest,42,16,4,1,44,11,4,4,13,19,32,48,80,22


In [26]:
spl_1980s['goals'] = spl_1980s.home_f + spl_1980s.away_f
epl_1980s['goals'] = epl_1980s.home_f + epl_1980s.away_f
spl_1980s['goals_perteam'] = spl_1980s.goals / spl_1980s.n_teams
epl_1980s['goals_perteam'] = epl_1980s.goals / epl_1980s.n_teams

In [27]:
spl_1980s[['year','goals_perteam']].groupby('year').sum()

,goals_perteam
year,
1980,48.400000
1981,48.100000
1982,49.800000
1983,52.900000
1984,53.300000
1985,52.200000
1986,49.800000
1987,55.833333
1988,53.833333
